In [1]:
import pandas as pd
from simple_salesforce import Salesforce
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
import datetime
from pytz import timezone
import boto3
from config import sf_username, sf_password, sf_security_token, aws_id, aws_secret

## Load and clean up data

In [2]:
# Authenticate to Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [3]:
# Pull Salesforce accounts with carbon reduction targets
crts = pd.DataFrame(sf.query_all("SELECT CRT_Name__c,CRT_Utility_Type__c,CRT_States__c,CRT_Final_Target_Text__c,CRT_Final_Target_Year__c,CRT_Interim_Targets_Summary__c,CRT_Mandatory_Voluntary__c FROM Account WHERE CRT_Final_Target_Text__c !=''")['records'])
del crts['attributes']
crts.head()

,CRT_Name__c,CRT_Utility_Type__c,CRT_States__c,CRT_Final_Target_Text__c,CRT_Final_Target_Year__c,CRT_Interim_Targets_Summary__c,CRT_Mandatory_Voluntary__c
0,Minnesota Power,Investor-Owned,MN,100% carbon-free energy by 2050,2050-01-01,50% reduction in carbon emissions by 2020; 80%...,Voluntary
1,Town of North Attleborough - (MA),Public Power,MA,Net-zero GHG emissions by 2050,2050-01-01,N/A,Mandatory
2,Modesto Irrigation District,Public Power,CA,100% renewable energy and zero-carbon resource...,2045-01-01,N/A,Mandatory
3,New York State Electric & Gas,Investor-Owned,NY,Zero GHG emissions by 2040,2040-01-01,N/A,Mandatory
4,Public Service Company of New Mexico,Investor-Owned,NM,100% emissions-free generation by 2040 (Volunt...,2040-01-01,N/A,Both


In [4]:
# Check length of df
len(crts)

370

In [5]:
# Rename columns
crts.columns = ["Utility/Entity", "Utility/Entity Type","State(s)*","Final Target Summary", "Attainment Year","Interim Target(s) Summary","Mand/Vol"]
crts.head()

,Utility/Entity,Utility/Entity Type,State(s)*,Final Target Summary,Attainment Year,Interim Target(s) Summary,Mand/Vol
0,Minnesota Power,Investor-Owned,MN,100% carbon-free energy by 2050,2050-01-01,50% reduction in carbon emissions by 2020; 80%...,Voluntary
1,Town of North Attleborough - (MA),Public Power,MA,Net-zero GHG emissions by 2050,2050-01-01,N/A,Mandatory
2,Modesto Irrigation District,Public Power,CA,100% renewable energy and zero-carbon resource...,2045-01-01,N/A,Mandatory
3,New York State Electric & Gas,Investor-Owned,NY,Zero GHG emissions by 2040,2040-01-01,N/A,Mandatory
4,Public Service Company of New Mexico,Investor-Owned,NM,100% emissions-free generation by 2040 (Volunt...,2040-01-01,N/A,Both


In [6]:
# Change interim target na values
crts.loc[(crts['Interim Target(s) Summary']=='N/A')&(crts['Mand/Vol']=='Mandatory'), 'Interim Target(s) Summary'] = '--'
crts.head()

,Utility/Entity,Utility/Entity Type,State(s)*,Final Target Summary,Attainment Year,Interim Target(s) Summary,Mand/Vol
0,Minnesota Power,Investor-Owned,MN,100% carbon-free energy by 2050,2050-01-01,50% reduction in carbon emissions by 2020; 80%...,Voluntary
1,Town of North Attleborough - (MA),Public Power,MA,Net-zero GHG emissions by 2050,2050-01-01,--,Mandatory
2,Modesto Irrigation District,Public Power,CA,100% renewable energy and zero-carbon resource...,2045-01-01,--,Mandatory
3,New York State Electric & Gas,Investor-Owned,NY,Zero GHG emissions by 2040,2040-01-01,--,Mandatory
4,Public Service Company of New Mexico,Investor-Owned,NM,100% emissions-free generation by 2040 (Volunt...,2040-01-01,N/A,Both


In [7]:
# Drop column
crts.drop('Mand/Vol',axis=1,inplace=True)
crts.head()

,Utility/Entity,Utility/Entity Type,State(s)*,Final Target Summary,Attainment Year,Interim Target(s) Summary
0,Minnesota Power,Investor-Owned,MN,100% carbon-free energy by 2050,2050-01-01,50% reduction in carbon emissions by 2020; 80%...
1,Town of North Attleborough - (MA),Public Power,MA,Net-zero GHG emissions by 2050,2050-01-01,--
2,Modesto Irrigation District,Public Power,CA,100% renewable energy and zero-carbon resource...,2045-01-01,--
3,New York State Electric & Gas,Investor-Owned,NY,Zero GHG emissions by 2040,2040-01-01,--
4,Public Service Company of New Mexico,Investor-Owned,NM,100% emissions-free generation by 2040 (Volunt...,2040-01-01,N/A


In [8]:
# Make date year only
crts['Attainment Year'] = pd.to_datetime(crts["Attainment Year"], errors='coerce').dt.year.apply(str).str.split('.').str[0]
crts.head()

,Utility/Entity,Utility/Entity Type,State(s)*,Final Target Summary,Attainment Year,Interim Target(s) Summary
0,Minnesota Power,Investor-Owned,MN,100% carbon-free energy by 2050,2050,50% reduction in carbon emissions by 2020; 80%...
1,Town of North Attleborough - (MA),Public Power,MA,Net-zero GHG emissions by 2050,2050,--
2,Modesto Irrigation District,Public Power,CA,100% renewable energy and zero-carbon resource...,2045,--
3,New York State Electric & Gas,Investor-Owned,NY,Zero GHG emissions by 2040,2040,--
4,Public Service Company of New Mexico,Investor-Owned,NM,100% emissions-free generation by 2040 (Volunt...,2040,N/A


## Paste data in formatted Excel template

In [9]:
#Load existing Excel workbook
file = 'Excel_Template.xlsx'
wb = openpyxl.load_workbook(filename=file)

In [10]:
#Navigate to correct sheet
ws = wb['Dataset']

In [11]:
# Turn dataframe into special object for excel
rows = dataframe_to_rows(crts,index=False)

In [12]:
# Loop through rows object and paste data into sheet
for r_idx, row in enumerate(rows, 1):
    for c_idx, value in enumerate(row, 1):
         ws.cell(row=r_idx, column=c_idx, value=value)

In [13]:
# Clear formatting on empty cells 
no_fill = openpyxl.styles.PatternFill(fill_type=None)
side = openpyxl.styles.Side(border_style=None)
no_border = openpyxl.styles.borders.Border(left=side, right=side, top=side, bottom=side)

start_row = len(crts)+2
for row in ws.iter_rows(min_row=start_row):
    for cell in row:
        cell.fill = no_fill
        cell.border = no_border

In [14]:
# Add last updated date to workbook too
tz = timezone('US/Eastern')
update_dt = "Last Updated Date/Time: " + datetime.datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S') + " ET"
print(update_dt)

Last Updated Date/Time: 2021-11-09 15:32:30 ET


In [15]:
ws = wb['Introduction']
ws.cell(row=19, column=8, value=update_dt)

<Cell 'Introduction'.H19>

In [16]:
# Save as
wb.save('SEPA Utility Carbon-Reduction Tracker Dataset.xlsx')

In [17]:
# Authenticate to AWS
s3 = boto3.client('s3', aws_access_key_id=aws_id, aws_secret_access_key=aws_secret)

In [18]:
# Push object to S3 Bucket
s3.upload_file('SEPA Utility Carbon-Reduction Tracker Dataset.xlsx', 'sepa-utility-crt', 
               'SEPA Utility Carbon-Reduction Tracker Dataset.xlsx')

In [19]:
# Change object permissions
s3 = boto3.resource('s3')
object_acl = s3.ObjectAcl('sepa-utility-crt','SEPA Utility Carbon-Reduction Tracker Dataset.xlsx')
response = object_acl.put(ACL='public-read')